# 1. RAG Básico - Implementación con Streamlit

## Objetivos de Aprendizaje
- Comprender los componentes fundamentales de un sistema RAG
- Implementar una búsqueda por palabras clave simple
- Generar respuestas basadas en contexto recuperado
- Construir una interfaz interactiva con Streamlit

## ¿Qué es RAG (Retrieval-Augmented Generation)?

RAG es una arquitectura que combina dos fases para generar respuestas:
1. **Retrieval (Recuperación)**: Busca y recupera información relevante de una base de conocimiento.
2. **Generation (Generación)**: Utiliza la información recuperada como contexto para que un LLM genere una respuesta precisa y fundamentada.

### Ventajas:
- **Reduce alucinaciones**: Las respuestas se basan en datos concretos.
- **Conocimiento actualizado**: La base de conocimiento puede actualizarse fácilmente.
- **Transparencia**: Se puede citar la fuente de la información.

### Componentes de nuestro RAG Básico:
- **Base de documentos**: Una lista de strings en memoria.
- **Mecanismo de retrieval**: Búsqueda simple por coincidencia de palabras clave.
- **Generador**: Un LLM (GPT-4o) que recibe el contexto y la pregunta.

# Instalación de dependencias
!pip install openai streamlit

In [1]:
# Celda 2: Importaciones y configuración
import os
from openai import OpenAI
import json

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [4]:
# Inicialización del cliente OpenAI
def initialize_client():
    client = OpenAI(
        base_url=os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com"),
        api_key=os.getenv("GITHUB_TOKEN")
    )
    return client

client = initialize_client()
print("✅ Cliente OpenAI inicializado")

✅ Cliente OpenAI inicializado


# Base de documentos

Cargamos la base de documentos que vamos a utilizar para el RAG. En este caso, vamos a utilizar una base de documentos que contiene información sobre la historia de Argentina.

In [12]:
documents = [
    "La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana.",
    "Los modelos de lenguaje grande (LLM) son sistemas de IA entrenados en enormes cantidades de texto para generar y comprender lenguaje natural.",
    "RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.",
    "LangChain es un framework que facilita el desarrollo de aplicaciones con modelos de lenguaje, proporcionando herramientas para cadenas y agentes.",
    "El prompt engineering es la práctica de diseñar instrucciones efectivas para obtener los mejores resultados de los modelos de IA."
]

print(f"📚 Base de documentos creada con {len(documents)} documentos:")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc[:60]}...")

📚 Base de documentos creada con 5 documentos:
1. La inteligencia artificial es una rama de la informática que...
2. Los modelos de lenguaje grande (LLM) son sistemas de IA entr...
3. RAG (Retrieval-Augmented Generation) combina la búsqueda de ...
4. LangChain es un framework que facilita el desarrollo de apli...
5. El prompt engineering es la práctica de diseñar instruccione...


# Función de recuperación simple

La función simple_retrieval() es una función que toma una pregunta y un índice, y devuelve una lista de documentos relevantes. En este caso, la función simplemente devuelve los primeros 3 documentos del índice.

In [13]:
def simple_retrieval(query, documents):
    relevant_docs = []
    query_lower = query.lower()
    
    for doc in documents:
        if any(word in doc.lower() for word in query_lower.split()):
            relevant_docs.append(doc)
    
    return relevant_docs[:3]

# Prueba de la función de recuperación
test_query = "¿Qué es la inteligencia artificial?"
relevant_docs = simple_retrieval(test_query, documents)

print(f"🔍 Prueba de recuperación para: '{test_query}'")
print(f"Documentos relevantes encontrados: {len(relevant_docs)}")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. {doc}")

🔍 Prueba de recuperación para: '¿Qué es la inteligencia artificial?'
Documentos relevantes encontrados: 3
1. La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana.
2. Los modelos de lenguaje grande (LLM) son sistemas de IA entrenados en enormes cantidades de texto para generar y comprender lenguaje natural.
3. RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.


# Función de generación de respuesta
La función generate_response se encarga de generar una respuesta a partir de una pregunta y un contexto. Para ello, utiliza un modelo de lenguaje preentrenado y una plantilla de prompt.

In [14]:
def generate_response(client, query, context):
    prompt = f"""Contexto:
{context}

Pregunta: {query}

Responde basándote únicamente en el contexto proporcionado. Si la información no está en el contexto, indica que no tienes esa información."""

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=500
    )
    
    return response.choices[0].message.content

print("✅ Función de generación definida")

✅ Función de generación definida


# Sistema RAG completo - Ejemplo 1

In [10]:
query1 = "¿Qué es la inteligencia artificial?"

print(f"❓ Pregunta: {query1}")
print("\n" + "="*50)

# Paso 1: Recuperación
relevant_docs = simple_retrieval(query1, documents)
print(f"📋 Documentos relevantes ({len(relevant_docs)}):")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. {doc}")

# Paso 2: Generación
if relevant_docs:
    context = "\n".join(relevant_docs)
    response = generate_response(client, query1, context)
    print(f"\n🤖 Respuesta:")
    print(response)
else:
    print("❌ No se encontraron documentos relevantes")

❓ Pregunta: ¿Qué es la inteligencia artificial?

📋 Documentos relevantes (3):
1. La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana.
2. Los modelos de lenguaje grande (LLM) son sistemas de IA entrenados en enormes cantidades de texto para generar y comprender lenguaje natural.
3. RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.

🤖 Respuesta:
La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana.


# Sistema RAG completo - Ejemplo 2

In [16]:
query2 = "¿ Cómo funciona RAG ?"

print(f"❓ Pregunta: {query2}")
print("\n" + "="*50)

# Paso 1: Recuperación
relevant_docs = simple_retrieval(query2, documents)
print(f"📋 Documentos relevantes ({len(relevant_docs)}):")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. {doc}")

# Paso 2: Generación
if relevant_docs:
    context = "\n".join(relevant_docs)
    response = generate_response(client, query2, context)
    print(f"\n🤖 Respuesta:")
    print(response)
else:
    print("❌ No se encontraron documentos relevantes")

❓ Pregunta: ¿ Cómo funciona RAG ?

📋 Documentos relevantes (1):
1. RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.

🤖 Respuesta:
RAG (Retrieval-Augmented Generation) funciona combinando la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.


# Sistema RAG completo - Ejemplo 3 (pregunta sin contexto)

In [17]:
query3 = "¿Cuál es la capital de Francia?"

print(f"❓ Pregunta: {query3}")
print("\n" + "="*50)

# Paso 1: Recuperación
relevant_docs = simple_retrieval(query3, documents)
print(f"📋 Documentos relevantes ({len(relevant_docs)}):")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. {doc}")

# Paso 2: Generación
if relevant_docs:
    context = "\n".join(relevant_docs)
    response = generate_response(client, query3, context)
    print(f"\n🤖 Respuesta:")
    print(response)
else:
    print("❌ No se encontraron documentos relevantes")
    print("🤖 Respuesta: No tengo información sobre esa pregunta en mi base de documentos.")

❓ Pregunta: ¿Cuál es la capital de Francia?

📋 Documentos relevantes (3):
1. La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana.
2. Los modelos de lenguaje grande (LLM) son sistemas de IA entrenados en enormes cantidades de texto para generar y comprender lenguaje natural.
3. RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.

🤖 Respuesta:
Según el contexto proporcionado, no tengo información sobre cuál es la capital de Francia.
